In [1]:
from ipyleaflet import Map, Marker, MarkerCluster, LayerGroup, LayersControl, basemaps, basemap_to_tiles
from ipywidgets import HTML
import pandas as pd
import glob
import os
import re

# Resources
zip_to_coordinates = pd.read_csv('./resources/zip_to_coordinates.csv', dtype={'Zip/Postal Code': 'str'})

# Concatenate CSV files in ./input 
joined_files = os.path.join("./input", "*.csv")
joined_list = glob.glob(joined_files)
contacts = pd.concat(map(pd.read_csv, joined_list))

def reduceZipCode(zip):
    matches = re.search("^(\d{4,5})", str(zip))
    if matches:
        return matches.group(1)
    return None

contacts['Zip/Postal Code'] = contacts['Zip/Postal Code'].apply(reduceZipCode)
contacts = pd.merge(contacts, zip_to_coordinates, on='Zip/Postal Code')

In [2]:
m = Map(
    basemap=basemaps.CartoDB.Positron,
    center=(39, -95),
    zoom=4
)

markers = []
for i, row in contacts.iterrows():
    markers.append(Marker(
        location=(row.LAT, row.LNG),
        popup=HTML(f"<div>{row['First Name']} {row['Last Name']}<br />{row['Email']}</div><div><small>{row['Chapter']} {row['Status']}</small></div><div><small>{row['City']}, {row['State']}</small></div>"),
        draggable=False
    ))

marker_cluster = MarkerCluster(
    name="Clustered Markers",
    markers=markers,
    max_cluster_radius=1
)

m.add_layer(marker_cluster)

m

Map(center=[39, -95], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…